In [1]:
## TD Ameritrade auto-trading algo for ProfitMonk
import time
import urllib
import requests
from splinter import Browser
from config import client_id, account_password,account_username
import datetime
import pandas as pd
import json

import tdameritrade as td
from tdameritrade import TDClient 
import selenium
import tdameritrade.auth
from selenium import webdriver
from datetime import datetime, timezone

from datetime import date
from datetime import timedelta
from dateutil.relativedelta import relativedelta, FR
from datetime import datetime

# get the target expiry date   
v_friday = (date.today() + timedelta(days=6) + relativedelta(weekday=FR(+1))).strftime("%Y-%m-%d")  

# Symbol to Trade
v_symbol = 'SPY'

#create an option order
def placeOptionOrder(v_orderType,v_price, v_duration,v_instruction,v_quantity,v_symbol):
    #define headers
    header = {'Authorization': "Bearer {}".format(access_token),
             'Content-Type':"application/json"}
    #define the endpoint for Saved orders, including your account ID
    endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders".format(account_id)

    #define the payload, in JSON format

    payload =   {
      "orderType": v_orderType,
      "session": "NORMAL",
      "price": v_price,
      "duration": v_duration,
      "orderStrategyType": "SINGLE",
      "orderLegCollection": [
        {
          "instruction": v_instruction,
          "quantity": v_quantity,
          "instrument": {
            "symbol": v_symbol,
            "assetType": "OPTION"
              }
            } 
          ]
        }
    #make a post 
    content = requests.post(url = endpoint, json = payload, headers = header)
    #print(content)

    return content.status_code
    #content.json()
    
#create an order
def PlaceStockOrder(v_orderType,v_duration,v_instruction,v_quantity,v_symbol,v_assetType):
    #define headers
    header = {'Authorization': "Bearer {}".format(access_token),'Content-Type':"application/json"}

    #define the endpoint for Saved orders, including your account ID
    endpoint = r"https://api.tdameritrade.com/v1/accounts/{}/savedorders".format(account_id)
    
    #define the payload, in JSON format
    payload =  { 'orderType': v_orderType
                ,'session': 'NORMAL'
                ,'duration': v_duration
                ,'orderStrategyType': 'SINGLE'
                ,'orderLegCollection': [
                        {'instruction': v_instruction,'quantity': v_quantity,'instrument': 
                         {'symbol': v_symbol, 'assetType': v_assetType } } ] }

    #make a post 
    content = requests.post(url = endpoint, json = payload, headers = header)

    # 
    return content.status_code


# define the path to chrome driver
executable_path = {'executable_path' : r'C:\Users\sakthivel.gounder\Desktop\Project\TDAmeritrade\chromedriver\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless= False)

method = 'GET'
url = 'https://auth.tdameritrade.com/auth?'
client_code = client_id + '@AMER.OAUTHAP'
payload = {'response_type':'code', 'redirect_uri': 'http://localhost:8080', 'client_id':client_code}

#build url
build_url = requests.Request(method, url,params = payload).prepare()
built_url = build_url.url

#print(built_url)
#go to url
browser.visit(built_url)
#define the emlemts to pass through the form
payload = {'username':account_username, 'password':account_password}

#fill out each element in the form
browser.find_by_id("username").first.fill(payload['username'])
browser.find_by_id("password").first.fill(payload['password'])
browser.find_by_id("accept").first.click()

#give it a sec for next page
time.sleep(1)

#click to accept conditions and allow
browser.find_by_id("accept").first.click()

#give it a second
time.sleep(1)
new_url = browser.url

#grab url and parse it
parse_url = urllib.parse.unquote(new_url.split('code=')[1])
browser.quit()

#define the endpoint
url = r'https://api.tdameritrade.com/v1/oauth2/token'

#define the headers
headers ={'Content-Type': "application/x-www-form-urlencoded"}

#define the payload
payload={'grant_type':'authorization_code'
        ,'access_type':'offline'
        ,'code': parse_url
        ,'client_id':client_id
        ,'redirect_uri':'http://localhost:8080'
        }

authReply = requests.post(url, headers = headers, data = payload)


#convert json string to dictionary

decoded_content = authReply.json()
access_token = decoded_content['access_token']
headers = {'Authorization' : "Bearer {}".format(access_token)}


## access single account 
acct1_endpoint = r"https://api.tdameritrade.com/v1/accounts"
acct1_content = requests.get(url = acct1_endpoint, headers = headers)
acct_info = acct1_content.json()
#convert to data dictionary
account_id = acct1_content.json()[0]['securitiesAccount']['accountId']
pos_endpoint = r"https://api.tdameritrade.com/v1/accounts/{}?fields=positions".format(account_id)
pos_content  =  requests.get(url = pos_endpoint,  headers = headers)
## get the position in dataframe
securitiesAccount = json.loads(pos_content.text)
recs = []

positions = securitiesAccount['securitiesAccount']['positions']
for rows in positions:
    rows['accountId'] = securitiesAccount['securitiesAccount']['accountId']
    rows['assetType'] = rows['instrument']['assetType']
    rows['cusip'] = rows['instrument']['cusip']
    rows['symbol'] = rows['instrument']['symbol']
    
    recs.append(rows)
pos_df = pd.DataFrame(recs)
pos_df = pos_df[['accountId','assetType','symbol','cusip','settledLongQuantity','marketValue','settledShortQuantity']]

#get the quote

header = {'Authorization': "Bearer {}".format(access_token),
         'Content-Type':"application/json"}
#define the endpoint for Saved orders, including your account ID
endpoint = r"https://api.tdameritrade.com/v1/marketdata/chains?symbol={}&contractType=PUT&strikeCount=50&fromDate={}&toDate={}".format(v_symbol,v_friday,v_friday)
endpoint1 = r"https://api.tdameritrade.com/v1/marketdata/{}/quotes".format(v_symbol)

#make a post 
res = requests.get(url = endpoint, headers = header)
spy_quote = json.loads(res.content)

res1 = requests.get(url = endpoint1, headers = header)
SPYCurrPrice = json.loads(res1.content)[v_symbol]['bidPrice']
#Premiumtocollect = SPYCurrPrice*0.015
## convert the option quote to dataframe
spy_quotes =  spy_quote["putExpDateMap"]
df = pd.DataFrame(columns=['ExpiryDate','StrikePrice','OptionSymbol','Bid','Ask','inTheMoney'])
bid=[]
ask=[]
opsym=[]
exdate=[]
stprice=[]
itm=[]
for row in spy_quotes:
    for rows in spy_quotes[row]:
        for d in spy_quotes[row][rows]:
            for k, v in d.items(): 
                if k in ('bid'):
                    bid.append(v)
                elif k in ('ask'):
                    ask.append(v)
                elif k in ('symbol'):
                    opsym.append(v)
                elif k in ('inTheMoney'):
                    itm.append(v)
                    exdate.append(row)
                    stprice.append(rows)
                    #print(k,v)
                
data = {'ExpiryDate':exdate,'StrikePrice':stprice,'OptionSymbol':opsym ,'Bid':bid, 'Ask':ask,'inTheMoney':itm}   
SPYOptionQuote = pd.DataFrame(data) 


In [2]:
import pandas as pa
import re
import string
from operator import itemgetter
#######################################################################
#LOAD THE HTML and Convert into DataFrame
#######################################################################
Read_HTML = pa.read_html('C:\\Users\\sakthivel.gounder\\Desktop\\Project\\TDAmeritrade\\Monk\\formatForParsing.html', header=0)

# Cover the list to Panda Dataframe

Dataset= Read_HTML[0]

#print(Dataset)

########################################################################
#Split the Idea colums to array
########################################################################

Idea_DataSet=(Dataset['Idea'].values)
#print(Idea_DataSet)
Idea_DataSetC=[]
Ex_date=[]

for x in Idea_DataSet:
    #res=re.sub('['+string.punctuation+']', '', x).split()
    res=x.split()
    #print(res)
    Revised_Data=(itemgetter(0,1,5,11,30,31,32)(res)) # pick the data based on their index and create a new directory
    Ex_date=[]
    Ex_date.append(Revised_Data[4]) #,Revised_Data[5],Revised_Data[6]])
    Ex_date.append(Revised_Data[5])
    Ex_date.append(Revised_Data[6])
    #print(Ex_date)
    Ex_date1=Ex_date[0]+Ex_date[1]+Ex_date[2]
    #print(xx)
   # print(Revised_Data[31]+Revised_Data[32])
    Revised_Data_again=({'CallType':Revised_Data[0],
                            'No_Of_Share':Revised_Data[1],
                            'ShareName':Revised_Data[2],
                            'SharePrice':Revised_Data[3],
                            'Exp_Date':Ex_date1})
    Idea_DataSetC.append(Revised_Data_again)
#print(Idea_DataSetC)
##########################################################################
#conver into DataFrame
Xdata=pa.DataFrame(Idea_DataSetC)
#print(Xdata)
#Add into orginal Data set
###########################################################################
Dataset['Call_Type']=Xdata['CallType']
Dataset['No_Of_Share']=Xdata['No_Of_Share']
Dataset['ShareName']=Xdata['ShareName']
Dataset['SharePrice']=Xdata['SharePrice']
Dataset['Exp_Date']=Xdata['Exp_Date']
#print(Dataset.columns)
#print(Dataset.head(10))

In [4]:
df_copy = Dataset[['Date of idea', 'Option symbol', 'Underlying stock price', 'Target sell price for option', 'Target stop loss price for option','Call_Type','No_Of_Share','ShareName','SharePrice','Exp_Date']]
df_copy.rename(columns={'Date of idea':'Date', 'ShareName':'Symbol','SharePrice':'SharePrice'
                  ,'Option symbol':'Option_Symbol','Underlying stock price':'Underlying_Price'
                  ,'Target sell price for option':'Sell_Price' ,'Target stop loss price for option':'StopLoss_Options'
                  }, inplace=True)

In [5]:
df_copy

,Date,Option_Symbol,Underlying_Price,Sell_Price,StopLoss_Options,Call_Type,No_Of_Share,Symbol,SharePrice,Exp_Date
0,"Nov 29, 2019",NDAQ200117C00110000,$104.8,$1.63,$0.03,Buy,6,NDAQ,$0.65,"Jan17,2020."
1,"Nov 29, 2019",CVS200110C00079000,$75.27,$1.78,$0.04,Buy,5,CVS,$0.71,"Jan10,2020."
2,"Nov 29, 2019",CME200110C00212500,$202.73,$3,$0.06,Buy,3,CME,$1.2,"Jan10,2020."
3,"Nov 29, 2019",PM200110C00087000,$82.93,$1.08,$0.02,Buy,9,PM,$0.43,"Jan10,2020."
4,"Nov 29, 2019",YUM200110C00106000,$100.67,$1.95,$0.04,Buy,5,YUM,$0.78,"Jan10,2020."
5,"Nov 29, 2019",TXN200110C00126000,$120.21,$2.95,$0.06,Buy,3,TXN,$1.18,"Jan10,2020."
6,"Nov 29, 2019",MCD200110C00205000,$194.48,$1.65,$0.03,Buy,6,MCD,$0.66,"Jan10,2020."
7,"Nov 29, 2019",ES200117C00085000,$82.64,$1.7,$0.03,Buy,5,ES,$0.68,"Jan17,2020."
8,"Nov 29, 2019",CLX200117C00155000,$148.23,$3.13,$0.06,Buy,3,CLX,$1.25,"Jan17,2020."
9,"Nov 29, 2019",BXP200117C00145000,$138.54,$1.63,$0.03,Buy,6,BXP,$0.65,"Jan17,2020."


In [46]:
#Profit Monk load

for i in range (len(df_copy)):
    retsell = placeOptionOrder('LIMIT',float(df_copy['Sell_Price'])-0.1, 'DAY','BUY_TO_OPEN',float(df_copy['No_Of_Share']),df_copy['Option_Symbol'])
    if retsell in (200,201):
        #update the df with success transaction details
        print('Success')
    elif retsel in (400,401):
        # update the df with failure transation 
        print('Failure')

TypeError: cannot convert the series to <class 'float'>

In [42]:
df_copy.head(5)

,Date,Option_Symbol,Underlying_Price,Sell_Price,StopLoss_Options,Call_Type,No_Of_Share,Symbol,SharePrice,Exp_Date
0,"Nov 29, 2019",NDAQ200117C00110000,$104.8,$1.63,$0.03,Buy,6,NDAQ,$0.65,"Jan17,2020."
1,"Nov 29, 2019",CVS200110C00079000,$75.27,$1.78,$0.04,Buy,5,CVS,$0.71,"Jan10,2020."
2,"Nov 29, 2019",CME200110C00212500,$202.73,$3,$0.06,Buy,3,CME,$1.2,"Jan10,2020."
3,"Nov 29, 2019",PM200110C00087000,$82.93,$1.08,$0.02,Buy,9,PM,$0.43,"Jan10,2020."
4,"Nov 29, 2019",YUM200110C00106000,$100.67,$1.95,$0.04,Buy,5,YUM,$0.78,"Jan10,2020."
